In [1]:
import cutde.fullspace as FS
import numpy as np
import matplotlib
%matplotlib widget
import matplotlib.pyplot as plt
import pandas as pd
import os 
from scipy.linalg import svd
from operator import itemgetter
import plotly.graph_objects as go
from scipy.signal import resample, detrend, resample_poly
from scipy.integrate import cumulative_trapezoid
import matplotlib.dates as mdates
from datetime import datetime, timedelta
from operator import itemgetter

In [2]:

wellcols=np.arange(12)
WellGeom=pd.read_excel('/home/spri902/Collab_metadata/Well_Points.xlsx',header=0,usecols=wellcols)
WellGeom.columns = WellGeom.columns.str.replace(' ','')
WellGeom.x = WellGeom.x/3.28084
WellGeom.y = WellGeom.y/3.28084
WellGeom.z = WellGeom.z/3.28084

In [4]:

os.chdir('/home/spri902/EGS_Collab/4850/fractures/')
with open('frac_plane_points.npy','rb') as f:
    x = np.load(f)
otp = x[0:80,:]
# otp = np.array(itemgetter(19,39,59)(x))
ctr = otp.mean(axis=0)

In [5]:
mwells=[]
wellList = ['E1-OB','E1-OT','E1-PDT','E1-PDB','E1-PST','E1-PSB']
for i in wellList:
    tmpwell = WellGeom[WellGeom["HoleID"]== i]
    tmpwell = tmpwell.iloc[:,0:6]
    mwells.append(tmpwell)
mwells = pd.concat(mwells)

swells=[]
wellList = ['E1-I','E1-P']
for i in wellList:
    tmpwell = WellGeom[WellGeom["HoleID"]== i]
    tmpwell = tmpwell.iloc[:,0:6]
    swells.append(tmpwell)
swells = pd.concat(swells)

# map DAS channel data to actual well coordinates
PDTchans = np.r_[1271:1388]
PDBchans = np.r_[1091:1210]
PDBpnts = mwells[mwells['HoleID'] == 'E1-PDB' ].iloc[10::10,:] # take every 10th point (1 meter spacing along well)
PDTpnts = mwells[mwells['HoleID'] == 'E1-PDT' ].iloc[10::10,:]

In [ ]:
# fig=plt.figure()
# ax = fig.add_subplot(projection='3d')
# ax.scatter(x[80:,0],x[80:,1],x[80:,2],c='black')
# ax.scatter(x[0:80,0],x[0:80,1],x[0:80,2],c='red')
# ax.scatter(x[0,0],x[0,1],x[0,2],c='blue',s=40)
# ax.scatter(x[19,0],x[19,1],x[19,2],c='blue',s=40)
# ax.scatter(x[39,0],x[39,1],x[39,2],c='blue',s=40)
# ax.scatter(x[59,0],x[59,1],x[59,2],c='blue',s=40)

# s1=ax.scatter(mwells.x,mwells.y,mwells.z,\
#     label='Monitoring Wells',marker='.',color='black',s=2)
# s2=ax.scatter(swells[swells['HoleID'] == 'E1-I'].x, swells[swells['HoleID'] == 'E1-I'].y, swells[swells['HoleID'] == 'E1-I'].z,\
#     label='Injector',marker='.',color='green',s=2)
# s3=ax.scatter(swells[swells['HoleID']=='E1-P'].x, swells[swells['HoleID']=='E1-P'].y, swells[swells['HoleID']=='E1-P'].z,\
#     label='Producer',marker='.',color='red',s=2)

### Fit Plane to OTP connector x,y,z points

In [6]:
def planeFit(points):
    """
    p, n = planeFit(points)

    Given an array, points, of shape (d,...)
    representing points in d-dimensional space,
    fit an d-dimensional plane to the points.
    Return a point, p, on the plane (the point-cloud centroid),
    and the normal, n.
    """
    points = np.reshape(points, (np.shape(points)[0], -1)) # Collapse trailing dimensions
    assert points.shape[0] <= points.shape[1], "There are only {} points in {} dimensions.".format(points.shape[1], points.shape[0])
    ctr = points.mean(axis=1)
    x = points - ctr[:,np.newaxis]
    # M = np.dot(x, x.T) # Could also use np.cov(x) here.
    M = np.cov(x)
    return ctr, svd(M)[0][:,-1]


pctr, normvec = planeFit(otp.T)
fittedPlane = -pctr.dot(normvec)

### Now build points for fault plane and fault triangles 
Triangle vertices order:
- top left, 
- top right, 
- bottom right, 
- bottom left

In [36]:
# plot plane and 
xb = np.linspace(pctr[0]-10,pctr[0]+10,80)
yb = np.linspace(pctr[1]-10,pctr[1]+10,80)
# xb = otp[:,0]
# yb = otp[:,1]
xx,yy = np.meshgrid(xb,yb)
zz = -(normvec[0] * xx + normvec[1] * yy + fittedPlane) * 1. /normvec[2]
 
# points order top left, top right, bottom right, bottom left
fault_pts = np.array(
    (xx[-1,-1], yy[-1,-1], zz[-1,-1],
    xx[0,-1], yy[0,-1], zz[0,-1],
    xx[0,0], yy[0,0], zz[0,0],
    xx[-1,0], yy[-1,0], zz[-1,0]
    )).reshape(4,3)


fault_tris = np.array([[0, 1, 2], [0, 2, 3]], dtype=np.int64)

### Plot triangle mesh on fault plane

In [ ]:
plt.figure()
plt.triplot(fault_pts[:, 1], fault_pts[:, 2], fault_tris)
plt.xlabel("y")
plt.ylabel("z")
plt.show()

### Build observation points for TDE code

In [40]:
nobs = 50
xoff = [800,830]
yoff = [-1320,-1270]
zoff = [-220,440]
xs = np.linspace(*xoff,nobs)
ys = np.linspace(*yoff,nobs)
zs = np.linspace(*zoff,nobs)
obsx,obsy,obsz = np.meshgrid(xs,ys,zs)
pts = np.array([obsx,obsy,obsz]).reshape((3,-1)).T.copy()
nu=0.24

### Plot the observation points 

In [ ]:
fig=plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(obsx,obsy,obsz,alpha=0.01)
ax.plot_surface(xx, yy, zz,color='y',alpha=0.9,shade=True)
# ax.scatter(xx,yy,zz)
# ax.scatter(otp[:,0],otp[:,1],otp[:,2],c='r',marker='o')
# ax.plot(pctr[0],pctr[1],pctr[2],'.m')
s1=ax.scatter(mwells.x,mwells.y,mwells.z,\
    label='Monitoring Wells',marker='.',color='black',s=2)
s2=ax.scatter(swells[swells['HoleID'] == 'E1-I'].x, swells[swells['HoleID'] == 'E1-I'].y, swells[swells['HoleID'] == 'E1-I'].z,\
    label='Injector',marker='.',color='green',s=2)
s3=ax.scatter(swells[swells['HoleID']=='E1-P'].x, swells[swells['HoleID']=='E1-P'].y, swells[swells['HoleID']=='E1-P'].z,\
    label='Producer',marker='.',color='red',s=2)
ax.scatter(fault_pts[:,0],fault_pts[:,1],fault_pts[:,2],marker='.',color='magenta',s=30)


### Call the Full Space strain matrix function
Inputs:
- obs points
- fault corners and triangle vertices
- Poisson ratio

In [43]:
strain_mat = FS.strain_matrix(obs_pts=pts, tris=fault_pts[fault_tris], nu=0.24)
disp_mat = FS.disp_matrix(obs_pts=pts, tris=fault_pts[fault_tris], nu=0.24)

### The strain output array is a (Npts, 6,Ntri,3 ) array 
The 2nd dimension refers to the components of the strain tensor (6)
The 4th dimension refers to the components of the slip vector (3)

Strain tensor:
- 0 is xx
- 1 is yy 
- 2 is zz
- 3 is xy 
- 4 is xz 
- 5 is yz

Slip vector:
- 0 is strike slip
- 1 is dip slip
- 2 is tensile / opening

In [68]:
# Sum over the 3rd component of the slip vector for tensile 
strain = np.sum(strain_mat[:,:,:,2], axis=2)
# Reshape into a grid that is (npts,npts,6)
strain = strain.reshape((obsy.shape[0], obsy.shape[1], obsy.shape[2], 6))

### The displacement output array is a (Npts, 3,Ntri,3 ) array 
The 2nd dimension refers to the components of the displacement vector (3)
The 4th dimension refers to the components of the slip vector (3)

Displacement vector:
- 0 is x
- 1 is y 
- 2 is z


Slip vector:
- 0 is strike slip
- 1 is dip slip
- 2 is tensile / opening

In [64]:
# Sum over the 3rd component of the slip vector for tensile 
disp = np.sum(disp_mat[:,:,:,2], axis=2)
# Reshape into a grid that is (npts,npts,6)
disp = disp.reshape((obsy.shape[0], obsy.shape[1], obsy.shape[2], 3))

### Unpack the strain tensor and displacement vector into each component for plotting

In [65]:
strnList = [0,1,2,3,4,5]
c0,c1,c2,c3,c4,c5 = [*[strain[:,:,:, i] for i in strnList]]

dispList = [0,1,2]
d0,d1,d2 = [*[disp[:,:,:, i] for i in dispList]]

In [ ]:
comp = c0
fig = go.Figure(data=go.Volume(
    x=obsx.flatten(),
    y=obsy.flatten(),
    z=obsz.flatten(),
    value=comp.flatten(),
    isomin=np.min(comp)*0.25,
    isomax=np.max(comp)*0.25,
    opacity=0.1,
    # opacityscale='extremes', # needs to be small to see through all surfaces
    surface_count=20, # needs to be a large number for good volume rendering
    colorscale = 'RdBu_r'
    ))
fig.add_scatter3d(x=mwells.x, y=mwells.y, z=mwells.z,mode='markers',
  marker=dict(
        size=1,
        color='black',                # set color to an array/list of desired values
        # colorscale='Viridis',   # choose a colorscale
        opacity=0.8))
# fig.add_trace(go.Mesh3d(x=xx.flatten(),y=yy.flatten(),z=zz.flatten(),
#         opacity=0.5,
#         color='yellow'))
fig.update_layout(
  autosize=False,
  width=1000,
  height=1000,
)
fig.show()



In [ ]:
comp = d0
fig = go.Figure(data=go.Volume(
    x=obsx.flatten(),
    y=obsy.flatten(),
    z=obsz.flatten(),
    value=comp.flatten(),
    isomin=np.min(comp)*.25,
    isomax=np.max(comp)*.25,
    opacity=0.1, # needs to be small to see through all surfaces
    surface_count=17, # needs to be a large number for good volume rendering
    ))
fig.add_scatter3d(x=mwells.x, y=mwells.y, z=mwells.z,mode='markers',
  marker=dict(
        size=1,
        color='black',                # set color to an array/list of desired values
        # colorscale='Viridis',   # choose a colorscale
        opacity=0.8))
fig.update_layout(
  autosize=False,
  width=1000,
  height=1000,
)
fig.show()

In [61]:
df_fullB = pd.read_pickle('/home/spri902/EGS_Collab/4850/results/maystim/processed_DAS/lpFilter/wellPDB/maystim22_26_combined_full')
df_fullT = pd.read_pickle('/home/spri902/EGS_Collab/4850/results/maystim/processed_DAS/lpFilter/wellPDT/maystim22_26_combined_full')
dasScaler = 116.0
df_fullB = df_fullB.multiply(dasScaler)
df_fullT = df_fullT.multiply(dasScaler)
df_fullB = detrend(df_fullB,axis=0,type='linear')
df_fullT = detrend(df_fullT,axis=0,type='linear')

In [ ]:
wn     = ['OT','OB','PST','PSB','PDB','PDT']
nfile_list = sorted(os.walk('/data1/parker/EGS_iDAS'))
nfile_list = nfile_list[1:]
#file_list = file_list[1:]
nfile_list = [group[2] for group in nfile_list]
nfile_list = [item for sublist in nfile_list for item in sublist]
# [file_list.append(f) for f in nfile_list]
fd = [name.split("_") for name in nfile_list]
fl = [fd[file][2].split(".") for file in range(len(fd))]
fl = [el[0] for el in fl]
DASdates = [datetime.strptime(d,'%y%m%d%H%M%S') for d in sorted(fl)]
# these are files that get skipped during the low pass filtering process and so the dates need to be removed 
ind2rem = [0, 90, 91, 257, 258, 1571, 1572, 3082, 3083, 5085, 5086, 5599, 5600, 5961, 5962, 7623, 7624, 8841, 8842, 9562]
# remove in reverse so that the indices remain in the correct order for removal
for index in sorted(ind2rem,reverse=True):
    del DASdates[index]

In [ ]:
# Bring in Injection data
os.chdir('/home/spri902/EGS_Collab/4850/stimflow/')
injFiles = sorted(os.listdir('/home/spri902/EGS_Collab/4850/stimflow/'))
injDat = pd.concat((pd.read_csv(f,header=1,usecols=[0,2,4,24],\
                           parse_dates = [0],infer_datetime_format=True) \
                    for f in injFiles if f.endswith('.csv')),axis=0)
injDat.rename(columns={'hh:mm:ss':'date','LPM':'QLPM','LPM.1':'TLPM','psig.9':'psig'},inplace=True)
# tmp = pd.read_csv(injFiles[3],header=1,usecols=[0,16],\
                          # parse_dates=[0],infer_datetime_format=True)
# tmp.rename(columns={'hh:mm:ss':'date','psig.2':'psig'},inplace=True)
# injDat = pd.concat((injDat,tmp),axis=0)
injDat.reset_index(drop = True,inplace = True)
injDat.set_index('date',inplace=True)
injDat = injDat.iloc[14650:]
injDat = injDat[~injDat.index.duplicated(keep='first')]

In [ ]:
directory = sorted(os.walk('/home/spri902/mayCASSM'))
CASSMdates = [datetime.strptime(d,'%Y%m%d%H%M%S') for d in sorted(directory[0][1])]
dasdnums = mdates.date2num(DASdates)
cassmdnums = mdates.date2num(CASSMdates)
DASintB = resample(df_fullB,len(cassmdnums[:-2]))
DASintT = resample(df_fullT,len(cassmdnums[:-2]))
chansB=np.linspace(0,df_fullB.shape[1] - 1,df_fullB.shape[1]).astype(int)
chansT=np.linspace(0,df_fullT.shape[1] - 1,df_fullT.shape[1]).astype(int)
stimbeg = [96, 224, 352, 472, 507]
stimfin = [106, 232, 368, 475, 512]
stimbegLines = itemgetter(*stimbeg)(CASSMdates)
stimfinLines = itemgetter(*stimfin)(CASSMdates)
df_strain_intB = cumulative_trapezoid(DASintB,axis=0,initial=0)
df_strainB = cumulative_trapezoid(df_fullB,axis=0,initial=0) 
df_strain_intT = cumulative_trapezoid(DASintT,axis=0,initial=0)
df_strainT = cumulative_trapezoid(df_fullT,axis=0,initial=0)

In [ ]:
df_strainB_cut = df_strainB[7806,0:60]
df_strainT_cut = df_strainT[7806,0:60]

In [ ]:
fig, ax = plt.subplots(figsize=(14,6))
vm=np.nanpercentile(df_strainT,99)
# if vm < 0:
#     img=ax.pcolormesh(DASdates,chans,df_full.T,cmap='RdBu',vmin=2*vm,vmax=vm+abs(vm))
# if vm > 0:
#     img=ax.pcolormesh(DASdates,chans,df_full.T,cmap='RdBu',vmin=vm/2,vmax=vm*2)
img = ax.pcolormesh(DASdates,PDTpnts['Depth(m)'],df_strainT_cut.T,cmap='RdBu',vmin=-vm,vmax=vm)
ax.invert_yaxis()
ax.xaxis_date()


In [ ]:
comp = c0
fig = go.Figure()
fig.add_scatter3d(x=PDTpnts.x, y=PDTpnts.y, z=PDTpnts.z,mode='markers',
  marker=dict(
        size=10,
        color=comp.flatten(),                # set color to an array/list of desired values
        colorscale='RdBu_r',   # choose a colorscale
        opacity=0.8))
# fig.add_trace(go.Mesh3d(x=otp[:,0],y=otp[:,1],z=otp[:,2],
#         opacity=0.5,
#         color='yellow'))
fig.update_layout(
  autosize=False,
  width=1000,
  height=1000,
)
fig.show()